# Premier League Scraping

This code allows you to scrape Premier League match results since 1992 from TransferMarkt

In [1]:
import re
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def classement(rank): 
    lieu1 = rank.text.find("(")
    lieu2 = rank.text.find(".")
    text = rank.text
    text = text[lieu1+1:lieu2]
    return text

In [3]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import re

domicile = []
tag_domicile = []
ranking_domicile = []
score_domicile = []

exterieur = []
tag_exterieur = []
ranking_exterieur = []
score_exterieur = []

saison = []
game_tag = []
date = []
hour = []
competition = []


for annee in range(1992, 2024):
    dom = ext = sco = dat = hou = 0
    
    url = "https://www.transfermarkt.co.uk/premier-league/gesamtspielplan/wettbewerb/GB1/saison_id/" + str(annee)
    print(url)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    dom = soup.select('td[class="text-right no-border-rechts hauptlink"]')
    ext = soup.select('td[class="no-border-links hauptlink"]')
    sco = soup.select('td[class="zentriert hauptlink"]')
    dat = soup.select('td[class="hide-for-small"]')
    hou = soup.select('td[class="zentriert hide-for-small"]')

    for i in range(len(dom)):
        day = rank = 0

        # game_tag
        a_tag = sco[i].find('a')
        id_value = a_tag.get('id')
        game_tag.append(id_value)

        # team tag
        a_tag = dom[i].find('a')
        href_value = a_tag.get('href')
        tag_domicile.append(href_value.split('/')[4])
        a_tag = ext[i].find('a')
        href_value = a_tag.get('href')
        tag_exterieur.append(href_value.split('/')[4])
        
        domicile.append(dom[i].text.split('\xa0')[-1].strip())
        exterieur.append(ext[i].text.split('\xa0')[0].strip())
        competition.append(url.split('/')[6])
        
        score = (sco[i].text.split('\xa0')[1].strip())
        score_domicile.append(score[0:1])
        score_exterieur.append(score[2:3])
        saison.append(annee)        

        # date
        if annee < 2000:
            try:
                day = dat[i].text.strip()
                day = str(day[3:].strip())
                day = datetime.strptime(day, '%b %d, %Y')
                last_day = day.strftime('%d/%m/%Y')
            except ValueError:
                last_day = last_day
        else:
            day = dat[i].text.strip()
            if day == "":
                last_day = last_day
            else:
                last_day = re.search(r'\d+/\d+/\d+', day).group()        
        date.append(last_day)

        # hour
        heure = hou[i].text.strip()
        if heure != "":
            test = heure.find(":")
            last_hour = heure
        else:
            last_hour = last_hour
        hour.append(last_hour)
        
        ranking_domicile.append(classement(dom[i]))
        ranking_exterieur.append(classement(ext[i]))

https://www.transfermarkt.co.uk/premier-league/gesamtspielplan/wettbewerb/GB1/saison_id/1992
https://www.transfermarkt.co.uk/premier-league/gesamtspielplan/wettbewerb/GB1/saison_id/1993
https://www.transfermarkt.co.uk/premier-league/gesamtspielplan/wettbewerb/GB1/saison_id/1994
https://www.transfermarkt.co.uk/premier-league/gesamtspielplan/wettbewerb/GB1/saison_id/1995
https://www.transfermarkt.co.uk/premier-league/gesamtspielplan/wettbewerb/GB1/saison_id/1996
https://www.transfermarkt.co.uk/premier-league/gesamtspielplan/wettbewerb/GB1/saison_id/1997
https://www.transfermarkt.co.uk/premier-league/gesamtspielplan/wettbewerb/GB1/saison_id/1998
https://www.transfermarkt.co.uk/premier-league/gesamtspielplan/wettbewerb/GB1/saison_id/1999
https://www.transfermarkt.co.uk/premier-league/gesamtspielplan/wettbewerb/GB1/saison_id/2000
https://www.transfermarkt.co.uk/premier-league/gesamtspielplan/wettbewerb/GB1/saison_id/2001
https://www.transfermarkt.co.uk/premier-league/gesamtspielplan/wettbew

In [4]:
import csv
import os

nom_fichier = 'games.csv'

if os.path.exists(nom_fichier):
    os.remove(nom_fichier)

entetes = ['competition', 'game_tag', 'season', 'date', 'hour', 'domicile_team', 'domicile_tag', 'domicile_ranking', 'domicile_score', 'exterieur_team', 'exterieur_tag', 'exterieur_ranking', 'exterieur_score']
data = []
for i in range(len(domicile)):
    x = {
        'competition': competition[i],
        'game_tag': game_tag[i],
        'season': saison[i],
        'date': date[i],
        'hour': hour[i],
        'domicile_team': domicile[i],
        'domicile_tag': tag_domicile[i],
        'domicile_ranking': ranking_domicile[i],
        'domicile_score': score_domicile[i],
        'exterieur_team': exterieur[i],
        'exterieur_tag': tag_exterieur[i],
        'exterieur_ranking': ranking_exterieur[i],
        'exterieur_score': score_exterieur[i]
    }
    data.append(x)

with open(nom_fichier, mode='w', newline='', encoding='utf-8') as fichier_csv:
    writer = csv.DictWriter(fichier_csv, fieldnames=entetes)
    writer.writeheader()
    for element in data:
        writer.writerow(element)